In [8]:
import pandas as pd
import numpy as np
import warnings
import re
warnings.filterwarnings("ignore")

In [9]:
pd.options.display.max_columns = 50

In [10]:
pd.options.display.max_rows = 1000

In [11]:
# подгрузка всех файлов без modes2017
import os
dr = 'modes'
file_names = os.listdir(dr)

In [34]:
# файлы в список
mode_files = []
for name in file_names[1:]:
    mode_files.append(pd.read_excel(dr+'\\'+name,skiprows=1))
    print(name)

сводка 1 и 2 квартал 2017_v2.xls
сводка 3 квартал 2017.xls
сводка декабрь 2017_v2.xls
сводка ноябрь 2017_v2.xls
сводка октябрь 2017_v2.xls
сводная 2012_v2.xls
сводная 2013_v2.xls
сводная 2014_v2.xls
сводная 2015_v2.xls
сводная 2016_v2.xls


In [35]:
for df in mode_files:
#     for col in df:
#         print(col)
    if 603395 in df['№ партии.1']:
        print(df.shape)

In [36]:
del mode_files[7]['Unnamed: 42']

In [19]:
def rep_part(ls):
    for x in ls:
        x = str(x)
        x = x.replace('.0', '')
    return ls

# for mode in mode_files:
#     mode['№ партии'] = mode['№ партии.1'].apply(rep_part)
#     mode = mode[mode['№ партии'] != 'nan']
#     print(mode['№ партии']);

In [37]:
df_part = []
for mode in mode_files:
    for x in mode['№ партии.1']:
        for ch in ['.0']:
            df_part.append(str(x).replace(ch, ''))

df_part = rep_part(df_part)

# df_part.to_excel('df_part.xlsx')

In [38]:
len(df_part)

129559

In [39]:
# for x in df_part:
#     print(x)

In [40]:
header = mode_files[0].columns

In [41]:
# убираем заголовки
tmp = []
for mode in mode_files:
    mode.columns = range(mode.shape[1])
    tmp.append(mode)
mode_files = tmp

# убираем последние пять столбцов из всех таблиц
for mode in mode_files:
    mode.drop(mode.columns[37:43], axis=1, inplace=True)
    
mode_files[5].shape[1]

37

In [42]:
for i in mode_files:
    print(i.shape)

(9682, 37)
(5436, 37)
(2437, 37)
(2170, 37)
(2101, 37)
(19689, 37)
(19991, 37)
(18277, 37)
(25074, 37)
(24702, 37)


In [43]:
df_modes = pd.DataFrame()
# слияние всех режимов
for df in mode_files:
    df_modes = pd.concat([df_modes,df])

df_modes.shape

(129559, 37)

In [44]:
df_modes.shape

(129559, 37)

In [45]:
len(header)

42

In [46]:
df_modes.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36],
           dtype='int64')

In [47]:
df_modes.columns = header[:37]

In [48]:
df_modes['№ партии'] = df_part

In [49]:
df_modes = df_modes[df_modes['№ партии'] != 'nan']
df_modes.shape

(129086, 37)

In [50]:
df_modes.reset_index(drop=True, inplace=True)

In [51]:
df_modes.columns

Index(['№ пакета', '№ партии', 'плавка', 'НД (гост, ту)', 'марка стали',
       'кол-во, шт.', 'D/S', 'Дата термообработки', 'время начала т/о',
       'шаг балок закалочная/отпускная печи, сек', '1 зона по ВТР закалка',
       '1 зона уставка закалка', '1 зона фактическая температура закалка',
       '2 зона по ВТР закалка', '2 зона уставка закалка',
       '2 зона фактическая температура закалка', '3 зона по ВТР закалка',
       '3 зона уставка закалка', '3 зона фактическая температура закалка',
       'Температура трубы на выгрузке из печи закалки',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', 'Расход воды на закалочный спрейер, м3',
       'время выдержки в закалочной ванне, сек.',
       't˚ C воды в закалочной ванне', 't˚ C трубы после ванны',
       '1 зона ВТР и уставка отпуск', '1 зона фактическая температура отпуск',
       '2 зона ВТР и уставка отпуск', '2 зона фактическая температура отпуск',
       '3 зона ВТР и уставка отпуск'

In [52]:
df_modes = df_modes.dropna(subset = ['№ партии'])

In [53]:
df_modes[df_modes['№ партии'] ==  '-[026.xls]лист1!$m$18']

,№ пакета,№ партии,плавка,"НД (гост, ту)",марка стали,"кол-во, шт.",D/S,Дата термообработки,время начала т/о,"шаг балок закалочная/отпускная печи, сек",1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,2 зона уставка закалка,2 зона фактическая температура закалка,3 зона по ВТР закалка,3 зона уставка закалка,3 зона фактическая температура закалка,Температура трубы на выгрузке из печи закалки,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,"Расход воды на закалочный спрейер, м3","время выдержки в закалочной ванне, сек.",t˚ C воды в закалочной ванне,t˚ C трубы после ванны,1 зона ВТР и уставка отпуск,1 зона фактическая температура отпуск,2 зона ВТР и уставка отпуск,2 зона фактическая температура отпуск,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,4 зона фактическая температура отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска


In [54]:
df_modes.head()

,№ пакета,№ партии,плавка,"НД (гост, ту)",марка стали,"кол-во, шт.",D/S,Дата термообработки,время начала т/о,"шаг балок закалочная/отпускная печи, сек",1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,2 зона уставка закалка,2 зона фактическая температура закалка,3 зона по ВТР закалка,3 зона уставка закалка,3 зона фактическая температура закалка,Температура трубы на выгрузке из печи закалки,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,"Расход воды на закалочный спрейер, м3","время выдержки в закалочной ванне, сек.",t˚ C воды в закалочной ванне,t˚ C трубы после ванны,1 зона ВТР и уставка отпуск,1 зона фактическая температура отпуск,2 зона ВТР и уставка отпуск,2 зона фактическая температура отпуск,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,4 зона фактическая температура отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска
0,402826400,612387,40600,ТУ3,38Г2С,50,"73х5,5",01.01.2017,20-20,24/24,850,850,853,850,NaN,NaN,860,849,849,848,1м/с,воздух,1693,НЕТ,22,НЕТ,640,640,575,NaN,630,631,569,NaN,614,614,621
1,402804863,612387,6F8132,ТУ 21,30Г2,50,"73,02Х5,51",01.01.2017,22-30,24/24,850,850,848,850,NaN,NaN,860,847,845,847,"0,63",75-85,1712,НЕТ,22,НЕТ,640,641,640,NaN,630,631,630,NaN,614,614,621
2,402806423,701002,6F8132,ТУ 21,30Г2,50,"73,02Х5,51",01.01.2017,22-50,24/24,850,850,853,850,NaN,NaN,860,847,847,847,"0,63",75-85,1719,НЕТ,22,НЕТ,640,642,640,NaN,630,631,630,NaN,614,615,621
3,402806443,701002,6F8132,ТУ 21,30Г2,50,"73,02Х5,51",01.01.2017,23-10,24/24,850,850,849,850,NaN,NaN,860,847,848,848,"0,63",75-85,1722,НЕТ,22,НЕТ,640,640,640,NaN,630,630,630,NaN,614,613,621
4,402806356,701002,6F8132,ТУ 21,30Г2,50,"73,02Х5,51",01.01.2017,23-30,24/24,850,850,847,850,NaN,NaN,860,847,845,н/д,"0,63",75-85,н/д,НЕТ,22,НЕТ,640,642,640,NaN,630,631,630,NaN,614,614,н/д


In [55]:
# df_modes[df_modes['№ партии'] == '127569']

In [56]:
# df_part[df_part['№ партии.1'] == 612387]

In [57]:
# dict_part = df_part.to_dict('records')
# dict_part

In [58]:
#список усредняемых столбцов
l1 = [u'1 зона фактическая температура закалка',
        u'3 зона фактическая температура закалка',
        u'1 зона фактическая температура отпуск',
        u'3 зона фактическая температура отпуск',
        u'5 зона фактическая температура отпуск']

In [59]:
try:
    df_modes[l1] = df_modes[l1].astype(float)
    df_modes = df_modes.groupby('№ партии')[l1].apply(lambda x: x.mean())
except:
    df_modes = df_modes.drop_duplicates('№ партии', keep = 'first')

In [60]:
#df1 - исходная таблица без пакетов
df_modes = df_modes.drop(['№ пакета'], axis=1)

In [61]:
df_modes.index = df_modes['№ партии']
df_modes.head()

,№ партии,плавка,"НД (гост, ту)",марка стали,"кол-во, шт.",D/S,Дата термообработки,время начала т/о,"шаг балок закалочная/отпускная печи, сек",1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,2 зона уставка закалка,2 зона фактическая температура закалка,3 зона по ВТР закалка,3 зона уставка закалка,3 зона фактическая температура закалка,Температура трубы на выгрузке из печи закалки,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,"Расход воды на закалочный спрейер, м3","время выдержки в закалочной ванне, сек.",t˚ C воды в закалочной ванне,t˚ C трубы после ванны,1 зона ВТР и уставка отпуск,1 зона фактическая температура отпуск,2 зона ВТР и уставка отпуск,2 зона фактическая температура отпуск,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,4 зона фактическая температура отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска
№ партии,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
612387,612387,40600,ТУ3,38Г2С,50,"73х5,5",01.01.2017,20-20,24/24,850,850,853,850,NaN,NaN,860,849,849,848,1м/с,воздух,1693,НЕТ,22,НЕТ,640,640,575,NaN,630,631,569,NaN,614,614,621
701002,701002,6F8132,ТУ 21,30Г2,50,"73,02Х5,51",01.01.2017,22-50,24/24,850,850,853,850,NaN,NaN,860,847,847,847,"0,63",75-85,1719,НЕТ,22,НЕТ,640,642,640,NaN,630,631,630,NaN,614,615,621
701003,701003,6F8354,ТУ3,38Г2С,32,"73х5,5",01.01.2017,02-00,24/24,850,850,847,850,NaN,NaN,860,847,845,851,1м/с,воздух,66,НЕТ,22,НЕТ,640,642,575,NaN,630,631,569,NaN,614,614,560
701004,701004,6F8354,ТУ3,38Г2С,50,"73х5,5",01.01.2017,02-40,24/24,850,850,847,850,NaN,NaN,860,847,845,848,1м/с,воздух,67,НЕТ,22,НЕТ,640,642,575,NaN,630,631,569,NaN,614,614,560
701005,701005,6F8357,ТУ3,38Г2С,50,"73х5,5",01.01.2017,05-00,24/24,850,850,847,850,NaN,NaN,860,847,845,852,1м/с,воздух,69,НЕТ,22,НЕТ,640,642,575,NaN,630,631,569,NaN,614,614,560


In [62]:
df_modes = df_modes[df_modes['плавка'] != 'nan']
df_modes.shape

(29263, 36)

In [63]:
df_modes.to_excel('16022018.xlsx')

In [64]:
df_modes.reset_index(drop=True, inplace=True)

In [65]:
df_modes[df_modes['№ партии'] ==  '-[026.xls]лист1!$m$18']

,№ партии,плавка,"НД (гост, ту)",марка стали,"кол-во, шт.",D/S,Дата термообработки,время начала т/о,"шаг балок закалочная/отпускная печи, сек",1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,2 зона уставка закалка,2 зона фактическая температура закалка,3 зона по ВТР закалка,3 зона уставка закалка,3 зона фактическая температура закалка,Температура трубы на выгрузке из печи закалки,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,"Расход воды на закалочный спрейер, м3","время выдержки в закалочной ванне, сек.",t˚ C воды в закалочной ванне,t˚ C трубы после ванны,1 зона ВТР и уставка отпуск,1 зона фактическая температура отпуск,2 зона ВТР и уставка отпуск,2 зона фактическая температура отпуск,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,4 зона фактическая температура отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска


In [66]:
def replace_all(x):
    x = str(x)
    x = x.replace(u'X', '/')
    x = x.replace(u'x', '/')
    x = x.replace(u'Х', '/')
    x = x.replace(u'х', '/')
    x = x.replace(',', '.')
    x = x.replace('\\', '/')
    x = x.replace('/-', '/0')
    x = x.replace('-/', '0/')
    x = x.replace('***', '0')
    x = x.replace('*', '0')
    x = x.replace('-', '/')
    x = x.replace('nan', '')
    return x

def convert_step0(x):
    return replace_all(x).split('/')[0]
   
def convert_step1(x):
    try:
        return replace_all(x).split('/')[1]
    except:
        return x

df_modes = df_modes[df_modes[u'шаг балок закалочная/отпускная печи, сек']!='***']
df_modes[u'шаг балок закалочная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
# df_modes[u'шаг балок закалочная печь, сек'].to_excel('check.xlsx') 
df_modes[u'шаг балок отпускная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)
#del  df_modes2[u'шаг балок закалочная/отпускная печи, сек']

In [67]:
df_modes[u'D/S'].head()

0        73х5,5
1    73,02Х5,51
2        73х5,5
3        73х5,5
4        73х5,5
Name: D/S, dtype: object

In [68]:
def split_d(x):
    x = str(x)
    x = x.replace(',', '.')
    return re.split('х|Х|x|X', x)[0]

def split_s(x):
    x = str(x)
    x = x.replace(',', '.')
    try:
        return re.split('х|Х|x|X', x)[1]
    except:
        return x

In [69]:
def rep_ds(x):
    x = str(x)
    x = x.replace('п', '')
    x = x.replace('П', '')
    x = x.replace('nan', '')
    x = x.replace(',', '.')
    
    return x

df_modes['диаметр'] = df_modes[u'D/S'].apply(split_d)
df_modes['толщина стенки'] = df_modes[u'D/S'].apply(split_s)
df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(rep_ds)
df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
df_modes['диаметр'] = df_modes['диаметр'].apply(rep_ds)

In [70]:
def rep(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u'ек', '')
    x = x.replace(u'\мс', '')
    x = x.replace(u'm/c', '')
    x = x.replace(u'М/С', '')
    x = x.replace(u'М\С', '')
    x = x.replace(u'nan', '')
    x = x.replace(u'м/с', '')
    x = x.replace(u'м\с', '')
    x = x.replace(u',', u'.')
    x = x.replace(u'м\\', '')
    x = x.replace(u'м/', '')
    x = x.replace(u'*****', u'')
    x = x.replace(u'****', u'')
    x = x.replace(u'***', u'')
    x = x.replace(u'29.03.2017', u'')
    try:
        y = x.split('-')
        x = (float(y[0])+float(y[1]))/2
    except:
        pass
    return x
    
    
df_modes[u'Скорость прохождения трубы через спрейер, м/с'] = df_modes[u'Скорость прохождения трубы через спрейер, м/с'].apply(rep)
# df_modes[u'Скорость прохождения трубы через спрейер, м/с'].to_excel('check_u.xlsx')

In [71]:
def rep_s(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u' ', '')
    x = x.replace(u'.', '')
    return x

df_modes[u'время выдержки в закалочной ванне, сек.'] = df_modes[u'время выдержки в закалочной ванне, сек.'].apply(rep_s)
# df_modes[u'время выдержки в закалочной ванне, сек.'].to_excel('check_t.xlsx')

In [72]:
df_modes.shape

(29262, 40)

In [73]:
df_modes[u'№ плавки'] = df_modes[u'плавка']
l2 = [
    'Дата термообработки', u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка',
      '2 зона уставка закалка', '2 зона фактическая температура закалка', '2 зона фактическая температура отпуск', 
      '4 зона фактическая температура отпуск']
df_modes = df_modes.drop(l2, axis = 1)
df_modes.shape


(29262, 32)

In [74]:
df_modes.shape

(29262, 32)

In [75]:
str_to_del = [
    3.75,
    4.7,
    4.18,
    4.85,
    4.78,
    4.56,
    3.71,
    4.7,
    4.63,
    4.65,
    4.52,
    1.57,
    3.87,
    3.89,
    2.5,
    1.91,
    2.98,
    4.6,
    4.71,
    2.36,
    4.39,
    4.68,
    1.82,
    2.74,
    2.81,
    3.61
    ]

for i in str_to_del:
    df_modes.drop(df_modes[df_modes['НД (гост, ту)'] == i].index, inplace=True, axis = 0)

In [76]:
df_modes.shape

(29248, 32)

In [77]:
df_modes.head()

,№ партии,"НД (гост, ту)",марка стали,"кол-во, шт.",1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,3 зона уставка закалка,3 зона фактическая температура закалка,Температура трубы на выгрузке из печи закалки,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,"Расход воды на закалочный спрейер, м3","время выдержки в закалочной ванне, сек.",t˚ C воды в закалочной ванне,t˚ C трубы после ванны,1 зона ВТР и уставка отпуск,1 зона фактическая температура отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,№ плавки
0,612387,ТУ3,38Г2С,50,850,850,853,850,860,849,849,848,1,воздух,1693,НЕТ,22,НЕТ,640,640,575,630,631,569,614,614,621,24,24,73,5.5,40600
1,701002,ТУ 21,30Г2,50,850,850,853,850,860,847,847,847,0.63,75-85,1719,НЕТ,22,НЕТ,640,642,640,630,631,630,614,615,621,24,24,73.02,5.51,6F8132
2,701003,ТУ3,38Г2С,32,850,850,847,850,860,847,845,851,1,воздух,66,НЕТ,22,НЕТ,640,642,575,630,631,569,614,614,560,24,24,73,5.5,6F8354
3,701004,ТУ3,38Г2С,50,850,850,847,850,860,847,845,848,1,воздух,67,НЕТ,22,НЕТ,640,642,575,630,631,569,614,614,560,24,24,73,5.5,6F8354
4,701005,ТУ3,38Г2С,50,850,850,847,850,860,847,845,852,1,воздух,69,НЕТ,22,НЕТ,640,642,575,630,631,569,614,614,560,24,24,73,5.5,6F8357


In [78]:
df_modes.drop(df_modes[df_modes['№ партии'].isnull()].index, inplace=True, axis = 0)

In [79]:
df_modes.shape

(29248, 32)

In [80]:
def rep_steel(x):
    x = str(x)
    x = x.replace('092ГС','09Г2С')
    x = x.replace('13 ХФА','13ХФА')
    x = x.replace('13хфа','13ХФА')
    x = x.replace('13ХФА ','13ХФА')
    x = x.replace('13хфа-3','13ХФА-3')
    x = x.replace('15Х13H2MБ','15Х13H2МБ')
    x = x.replace('15х5мфбч','15Х5МФБЧ')
    x = x.replace('18х3мфб','18Х3МФБ')
    x = x.replace('18Х3МФБ ','18Х3МФБ')
    x = x.replace('18х3ХМФБ','18Х3МФБ')
    x = x.replace('18хзмфб','18Х3МФБ')
    x = x.replace('18хмфб','18ХМФБ')
    x = x.replace('18ХМФБ ','18ХМФБ')
    x = x.replace('20А простая','20А-пр')
    x = x.replace('20А-ПР','20А-пр')
    x = x.replace('20 А','20А')    
    x = x.replace('20A','20А') 
    x = x.replace('20А ','20А')
    x = x.replace('20 АУ','20А-У')
    x = x.replace('20АУ','20А-У')
    x = x.replace('30Г2 ','30Г2')
    x = x.replace('30Г2.','30Г2')
    x = x.replace('30г2-2','30Г2-2')
    x = x.replace('30г2ф','30Г2Ф')
    x = x.replace('30г2','30Г2')
    x = x.replace('30хгма','30ХГМА')
    x = x.replace('30ХГМА1','30ХГМА-1')
    x = x.replace('30хма','30ХМА')
    x = x.replace('32г1','32Г1')
    x = x.replace('32г2','32Г2')
    x = x.replace('32х1мф-1','32Х1МФ-1')
    x = x.replace('32ха','32ХА')
    x = x.replace('37г2ф','37Г2Ф')
    x = x.replace('37г2Ф','37Г2Ф')
    x = x.replace('37Г2ф','37Г2Ф')
    x = x.replace('37Г2Ф ','37Г2Ф')
    x = x.replace('38Г2С ','38Г2С')
    x = x.replace('38г2с-4','38Г2С-4')
    x = x.replace('38Г2-С-9','38Г2С-9')
    x = x.replace('30Г2 ', '30Г2')
    return x

df_modes['марка стали'] = df_modes['марка стали'].apply(rep_steel)

In [81]:
def rep_vtr(x):
    x = str(x)
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('-','')
    x = x.replace('50/*','')
    x = x.replace('нет','')
    x = x.replace('НЕТ','')
    return x

df_modes['1 зона по ВТР закалка'] = df_modes['1 зона по ВТР закалка'].apply(rep_vtr)
df_modes['2 зона по ВТР закалка'] = df_modes['2 зона по ВТР закалка'].apply(rep_vtr)
df_modes['3 зона по ВТР закалка'] = df_modes['3 зона по ВТР закалка'].apply(rep_vtr)


In [82]:
def rep_nd(x):
    x = str(x)
    x = x.replace('н/д','')
    x = x.replace('.0', '')
    x = x.replace('НЕТ', '')
    x = x.replace('***', '')
    return x

df_modes['1 зона уставка закалка'] = df_modes['1 зона уставка закалка'].apply(rep_nd)
df_modes['1 зона фактическая температура закалка'] = df_modes['1 зона фактическая температура закалка'].apply(rep_nd)
df_modes['3 зона уставка закалка'] = df_modes['3 зона уставка закалка'].apply(rep_nd)
df_modes['3 зона фактическая температура закалка'] = df_modes['3 зона фактическая температура закалка'].apply(rep_nd)
df_modes['Температура трубы на выгрузке из печи закалки'] = df_modes['Температура трубы на выгрузке из печи закалки'].apply(rep_nd)
df_modes['Расход воды на закалочный спрейер, м3'] = df_modes['Расход воды на закалочный спрейер, м3'].apply(rep_nd)
df_modes['t˚ C воды в закалочной ванне'] = df_modes['t˚ C воды в закалочной ванне'].apply(rep_nd)
df_modes['1 зона фактическая температура отпуск'] = df_modes['1 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['3 зона ВТР и уставка отпуск'] = df_modes['3 зона ВТР и уставка отпуск'].apply(rep_nd)
df_modes['3 зона фактическая температура отпуск'] = df_modes['3 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['5 зона ВТР и уставка отпуск'] = df_modes['5 зона ВТР и уставка отпуск'].apply(rep_nd)
df_modes['5 зона фактическая температура отпуск'] = df_modes['5 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['Температура трубы на выгрузке из печи отпуска'] = df_modes['Температура трубы на выгрузке из печи отпуска'].apply(rep_nd)

In [83]:
def rep_u_sp(x):
    x = str(x)
    x = x.replace(',','.')
    x = x.replace('041','0.41')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')  
    x = x.replace('м','')
    x = x.replace('021', '0.21')
    x = x.replace('1.8', '0.18')
    x = x.replace('2.0', '0.2')
    x = x.replace('012', '0.12')
    if x == '45':
        return '0.45'
    return x

col = 'Скорость прохождения трубы через спрейер, м/с' 
df_modes[col] = df_modes[col].apply(rep_u_sp)

In [84]:
def rep_time(x):
    x = str(x)
    x = x.replace('nan','')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')
    x = x.replace('НОРМ','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('40-50','45')
    x = x.replace('сек','')
    x = x.replace('с','')
    x = x.replace(',','.')
    x = x.replace('11cек', '')
    return x


col ='время выдержки в закалочной ванне, сек.'
df_modes[col] = df_modes[col].apply(rep_time)

In [85]:
def rep_tempr(x):
    x = str(x)
    x = x.replace('******','')
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция', '')
    x = x.replace('\\','/')
    x = x.replace('-','/')
    x = x.replace('<', '')
    x = x.replace('>', '')
    x = x.replace('Доотпуск', 'доотпуск')
    x = x.replace('до', '')
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('БизнесТренд', '')
    x = x.replace('ВОЗДУХ', '')
    x = x.replace('Воздух', '')
    x = x.replace('воэдух', '')
    x = x.replace('высадка', '')
    x = x.replace('закалка', '')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('нет', '')
    x = x.replace('НОРМ', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('воздух', '')
    x = x.replace('61702', '')
    x = x.replace('опытные.','')
    x = x.replace(',','.')
    y = x.split('/')
    fl = True
    tmpy= []
    for i in y:
        if len(i) == 4:
            y.append(i[0:2])
            y.append(i[2:4])
            y.remove(i)
    for i in y:
        try:
            tmpy.append(float(i))
        except:
            tmpy.append(i)
            fl = False
    if(fl):
        y = tmpy   
        x = np.mean(y)
    if x=='0':
        x=''
    return x


col = 't˚ C трубы после спреера'
df_modes[col] = df_modes[col].apply(rep_tempr)
df_modes.drop(df_modes[df_modes[col]=='отпуск'].index, inplace = True, axis = 0)

In [86]:
def rep_tempr_bath(x):
    x = str(x)
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('бизнес тренд', '')
    x = x.replace('высадка.', '')
    x = x.replace('высадка', '')
    x = x.replace('ВЫСАДКА', '')
    x = x.replace('Высадка', '')
    x = x.replace('высадка', '') 
    x = x.replace('доотпуск.', 'доотпуск')
    x = x.replace('доотпуск', 'доотпуск')
    x = x.replace('Доотпуск', 'доотпуск')
    x = x.replace('ДООТПУСК', 'доотпуск')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('НК 8500', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('нормал-ция', '')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция.', '')
    x = x.replace('норм-ция', '')
    x = x.replace('нор-ция', '')
    x = x.replace('опытные.', '')
    x = x.replace('ОПЫТНЫЕ', '')
    x = x.replace('повторная', '')
    x = x.replace('повтор', '')
    x = x.replace('ПОВТОР', '')
    x = x.replace('Самара', '')
    x = x.replace('СИТЦ', '')
    x = x.replace('Трэнд', '')
    x = x.replace('ЧПТЗ', '')
    x = x.replace('ЧТПЗ', '')
    x = x.replace('30 СЕК.', '30')
    x = x.replace('2/я зак/ка', '')
    x = x.replace('ТРЕНД', '')
    x = x.replace('ПЛ.Р. Ц42/№10','')
    x = x.replace('Челябинск','')
    if x=='0':
        x=''
    if x=='.':
        x=''
    return x


df_modes['t˚ C трубы после ванны'] = df_modes['t˚ C трубы после ванны'].apply(rep_tempr)
df_modes['t˚ C трубы после ванны'] = df_modes['t˚ C трубы после ванны'].apply(rep_tempr_bath)
df_modes.drop(df_modes[df_modes['t˚ C трубы после ванны']=='отпуск'].index, inplace = True, axis = 0)

In [87]:
df_modes[df_modes['t˚ C трубы после ванны'] == 'ПЛ.Р. Ц42/№10']

,№ партии,"НД (гост, ту)",марка стали,"кол-во, шт.",1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,3 зона уставка закалка,3 зона фактическая температура закалка,Температура трубы на выгрузке из печи закалки,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,"Расход воды на закалочный спрейер, м3","время выдержки в закалочной ванне, сек.",t˚ C воды в закалочной ванне,t˚ C трубы после ванны,1 зона ВТР и уставка отпуск,1 зона фактическая температура отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,№ плавки


In [88]:
def rep_vtr(x):
    x = str(x)
    x = x.replace('н/д','')
    x = x.replace('***','')
    return x


df_modes['1 зона ВТР и уставка отпуск'] = df_modes['1 зона ВТР и уставка отпуск'].apply(rep_vtr)

In [89]:
def rep_vtr2(x):
    x = str(x)
    x = x.replace('***', '')
    x = x.replace('****', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('нет', '')
    if x == '67':
        return '670'
    if x == '69':
        return '690'
    return x

df_modes['2 зона ВТР и уставка отпуск'] = df_modes['2 зона ВТР и уставка отпуск'].apply(rep_vtr2)
df_modes['4 зона ВТР и уставка отпуск'] = df_modes['4 зона ВТР и уставка отпуск'].apply(rep_vtr2)

In [90]:
df_modes.shape

(29163, 32)

In [91]:
#186
df_modes.drop(df_modes[df_modes['шаг балок закалочная печь, сек'] == '0'].index, inplace=True, axis=0)
#204
df_modes.drop(df_modes[df_modes['шаг балок отпускная печь, сек'] == '0'].index, inplace=True, axis=0)

In [92]:
df_modes.shape

(28574, 32)

In [93]:
ls_mean = df_modes.columns[4:-3]
ls_mean

Index(['1 зона по ВТР закалка', '1 зона уставка закалка',
       '1 зона фактическая температура закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка', '3 зона уставка закалка',
       '3 зона фактическая температура закалка',
       'Температура трубы на выгрузке из печи закалки',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', 'Расход воды на закалочный спрейер, м3',
       'время выдержки в закалочной ванне, сек.',
       't˚ C воды в закалочной ванне', 't˚ C трубы после ванны',
       '1 зона ВТР и уставка отпуск', '1 зона фактическая температура отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '3 зона фактическая температура отпуск', '4 зона ВТР и уставка отпуск',
       '5 зона ВТР и уставка отпуск', '5 зона фактическая температура отпуск',
       'Температура трубы на выгрузке из печи отпуска',
       'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек'],
      dtype='object')

In [94]:
def get_mean(df):
    df = df.groupby(['№ плавки', '№ партии']).first()
    for col in ls_mean:
        if col in df:
            try:
                df[col] = df.groupby(['№ плавки', '№ партии'])[col].transform(lambda x: x.mean())
            except:
                pass
    df = df.reset_index()
    return df

In [95]:
df_modes['№ партии'] = df_modes['№ партии'].astype(str).str.lower()
df_modes['№ плавки'] = df_modes['№ плавки'].astype(str).str.lower()

In [96]:
df_modes.shape

(28574, 32)

In [97]:
tmp = get_mean(df_modes)
# tmp.groupby('№ партии').shape
tmp.shape

(28574, 32)

In [98]:
tmp[tmp['№ партии'].duplicated()]

,№ плавки,№ партии,"НД (гост, ту)",марка стали,"кол-во, шт.",1 зона по ВТР закалка,1 зона уставка закалка,1 зона фактическая температура закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,3 зона уставка закалка,3 зона фактическая температура закалка,Температура трубы на выгрузке из печи закалки,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,"Расход воды на закалочный спрейер, м3","время выдержки в закалочной ванне, сек.",t˚ C воды в закалочной ванне,t˚ C трубы после ванны,1 зона ВТР и уставка отпуск,1 зона фактическая температура отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,3 зона фактическая температура отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,5 зона фактическая температура отпуск,Температура трубы на выгрузке из печи отпуска,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки


In [99]:
tmp.to_excel('all_modes_clean_2.xlsx')